# OpenAI's Whisper

Whisper is an **A**utomatic **S**peech **R**ecognition (ASR) model from OpenAI. We use it to extract highly accurate text from YouTube videos.

In [5]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update -y && sudo apt install ffmpeg -y
!sudo apt install -y pydub AudioSegment


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-5au75bfp
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-5au75bfp
  Resolved https://github.com/openai/whisper.git to commit 3e1780fd37686666f568be9c99f5b5e3e4f2eb92
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 18.5 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230124-py3-none-any.whl size=1179407 sha256=13887b36f1e338f3b4511d6c391f46de86d3c5b1c070b2d4c9016fa260a1d0ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-shmjlqqr/wheels/a7/70/18/b7693c07b1d18b3dafb328f5d0496aa0d41a9c09ef332fd8e6
Successfully built opena

In [7]:
import whisper
import torch  # pytorch install steps: pytorch.org

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

model = whisper.load_model("large").to(device)

/usr/local/lib/python3.8/dist-packages/jax/__init__.py:177: UserWarning: The jax module appears to have been reloaded within the python process. This is not well-supported and can cause unpredictable side-effects. For information see https://github.com/google/jax/issues/13857.
  _warn("The jax module appears to have been reloaded within the python process. "


cpu


100%|█████████████████████████████████████| 2.87G/2.87G [00:59<00:00, 52.2MiB/s]


In [ ]:
from datasets import load_dataset

videos_meta = load_dataset(
    "jamescalam/channel-metadata",
    split="train"
)
videos_meta

Create videos metadata dictionary...

In [3]:

    }

NameError: ignored

In [2]:
# get list of MP3 audio files
paths = [str(x) for x in Path('./mp3').glob('*.mp3')]
print(len(paths))
print(paths[:5])

NameError: ignored

In [ ]:
# we get the IDs like so
paths[0].split('/')[-1][:-4]

'35Pdoyi6ZoQ'

In [ ]:
data = []
for i, path in enumerate(tqdm(paths)):
    _id = path.split('/')[-1][:-4]
    # transcribe to get speech-to-text data
    result = model.transcribe(path)
    # add results to data list
    data.extend(result['segments'])

In [ ]:
from pathlib import Path
from tqdm.auto import tqdm
import json

# set window (length of text chunk) and stride
window = 1
stride = 1  # smaller stride creates overlap

data = []

results = []
with open("transcription.jsonl", "w", encoding="utf-8") as fp:
    for i, path in enumerate(tqdm(paths)):
        _id = path.split('/')[-1][:-4]
        # transcribe to get speech-to-text data
        result = model.transcribe(path)
        segments = result['segments']
        # get the video metadata...
        video_meta = videos_dict[_id]
        for j in range(0, len(segments), stride):
            j_end = min(j+window, len(segments)-1)
            text = ''.join([x["text"] for x in segments[j:j_end]])
            start = segments[j]['start']
            end = segments[j_end]['end']
            row_id = f"{_id}-t{segments[j]['start']}"
            meta = {
                **video_meta,
                **{
                    "id": row_id,
                    "text": text.strip(),
                    "start": start,
                    "end": end
                }
            }
            data.append(meta)
            json.dump(meta, fp)
            fp.write('\n')

len(data)

100%|██████████| 108/108 [7:10:39<00:00, 239.26s/it] 


27214

---

## Append more to dataset without overwriting/redoing

First check what we already have

In [ ]:
import json

existing_ids = []

with open("transcription.jsonl", 'r', encoding='utf-8') as fp:
    for line in fp:
        obj = json.loads(line)
        existing_ids.append(obj['url'].split('/')[-1])

existing_ids = set(existing_ids)
len(existing_ids)

108

In [ ]:
list(existing_ids)[:5]

['gVAJ_l_S7uQ', '1gN1snKBLP0', 'YvVQgvAz9dY', '3Wqh4iUupbM', 'jjQetJtQDS4']

Get paths to videos not already in `existing_ids`...

In [ ]:
# load the audio file and play it
audio_file = AudioSegment.from_file("somewhere_over_the_rainbow.mp3", format="mp3")
play(audio_file)

ModuleNotFoundError: ignored

In [ ]:
paths = [x for x in paths if x.split('/')[-1][:-4] not in existing_ids]
print(len(paths))
print(paths[:5])

0
[]


In [ ]:
from pathlib import Path
from tqdm.auto import tqdm
import json

# set window (length of text chunk) and stride
window = 1
stride = 1  # smaller stride creates overlap

results = []
with open("transcription.jsonl", "a", encoding="utf-8") as fp:
    for i, path in enumerate(tqdm(paths)):
        _id = path.split('/')[-1][:-4]
        # transcribe to get speech-to-text data
        result = model.transcribe(path)
        segments = result['segments']
        # get the video metadata...
        video_meta = videos_dict[_id]
        for j in range(0, len(segments), stride):
            j_end = min(j+window, len(segments)-1)
            text = ''.join([x["text"] for x in segments[j:j_end]])
            start = segments[j]['start']
            end = segments[j_end]['end']
            _id = f"{_id}-t{segments[j]['start']}"
            meta = {
                **video_meta,
                **{
                    "id": _id,
                    "text": text.strip(),
                    "start": start,
                    "end": end
                }
            }
            json.dump(meta, fp)
            fp.write('\n')

len(data)